# Modelo de Identificação de Fraudes

- Informações da base:
    - 'Time' (tempo): contém os segundos decorridos entre cada transação e a primeira transação no conjunto de dados. 
    - 'Amount' (valor): é o valor da transação 
    - 'Class' (classe): é a variável de resposta e assume valor 1 em caso de fraude e 0 caso contrário.
    - "Infelizmente, devido a questões de confidencialidade, não podemos fornecer os recursos originais e mais informações básicas sobre os dados. Características V1, V2, … V28 são os principais componentes obtidos com PCA"

In [1]:
# Configuração do ambiente

# Garante que o notebook está na raiz do projeto
%cd .. 

# Verifica o diretório atual (Linux/Mac)
# !pwd  

# Verifica o diretório atual (Windows)
!cd 

C:\Users\flavi\Documents\GitHub\Projeto_4_Modelo_de_Identificacao_de_Fraude
C:\Users\flavi\Documents\GitHub\Projeto_4_Modelo_de_Identificacao_de_Fraude


In [2]:
# Importações necessárias 

import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime as dt
import xgboost as xgb

from scipy.stats import kstest, normaltest, anderson

from sklearn.preprocessing import PowerTransformer, MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree, metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.feature_selection import SelectKBest, f_classif

from sklearn.model_selection import train_test_split, GridSearchCV
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids, NearMiss 
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.combine import SMOTEENN

from sklearn.metrics import (
    confusion_matrix, 
    accuracy_score, 
    precision_score, 
    recall_score,
    precision_recall_curve,
)

import warnings
from src.config.paths import DADOS_CREDICARD, DADOS_CREDICARD_TRATADO
from src.config.auxiliares_ml import downcast_dataframe, testar_modelos_com_undersampling

warnings.filterwarnings('ignore')
sns.set_theme(palette="bright")

In [3]:
# Importando a base

transacoes = pd.read_parquet(DADOS_CREDICARD_TRATADO)

In [4]:
# Visualizando estatísticas descritivas da base 

transacoes.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.867188,1.339397e-08,0.000000,-4.800400e-08,6.643411e-09,2.657364e-08,-1.500125e-09,-1.071518e-09,-1.071518e-10,-3.214554e-10,...,-1.928732e-09,3.643161e-09,3.214554e-10,1.259033e-09,2.143036e-09,1.034684e-09,1.138488e-10,-1.272427e-10,88.349619,0.001727
std,47487.195312,1.958611e+00,1.651183,1.516203e+00,1.415832e+00,1.380180e+00,1.332214e+00,1.237020e+00,1.194202e+00,1.098575e+00,...,7.344701e-01,7.256873e-01,6.244100e-01,6.056016e-01,5.212620e-01,4.822155e-01,4.035722e-01,3.300574e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-72.715729,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-0.598550,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,0.065486,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097605e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,0.803724,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273458e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,22.057730,9.382559e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519588e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


### NOTA: Vamos melhorar a escala das features

In [5]:
# Lista de colunas a serem normalizadas (excluindo 'Class')

colunas_para_normalizar = ['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
                           'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
                           'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']

# Normalizando as colunas (min-max scaling)

transacoes[colunas_para_normalizar] = transacoes[colunas_para_normalizar] / transacoes[colunas_para_normalizar].max()

In [6]:
# Visualizando estatísticas descritivas da base normalizada

transacoes.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,0.548717,-9.115664e-11,2.556490e-12,6.094481e-12,-3.268473e-12,7.593905e-13,-2.706109e-12,-9.158887e-13,2.672787e-12,1.551733e-12,...,-4.199505e-13,3.833444e-12,-4.280388e-13,1.039828e-11,-9.602410e-13,-1.266194e-11,7.359425e-13,-2.957288e-13,0.003439,0.001727
std,0.274828,7.978622e-01,7.486303e-02,1.616036e-01,8.390162e-02,3.966036e-02,1.817519e-02,1.025872e-02,5.969613e-02,7.044774e-02,...,2.700174e-02,6.909410e-02,2.771879e-02,1.321061e-01,6.932269e-02,1.370997e-01,1.276825e-02,9.751983e-03,0.009736,0.041527
min,0.000000,-2.297724e+01,-3.296610e+00,-5.150577e+00,-3.367737e-01,-3.268329e+00,-3.568885e-01,-3.612026e-01,-3.659517e+00,-8.614345e-01,...,-1.280395e+00,-1.040945e+00,-1.988943e+00,-6.187363e-01,-1.369144e+00,-7.404875e-01,-7.138282e-01,-4.558666e-01,0.000000,0.000000
25%,0.313681,-3.749082e-01,-2.713561e-02,-9.489574e-02,-5.028876e-02,-1.987253e-02,-1.048129e-02,-4.594728e-03,-1.042773e-02,-4.123744e-02,...,-8.395997e-03,-5.163722e-02,-7.184099e-03,-7.734374e-02,-4.217585e-02,-9.296326e-02,-2.240892e-03,-1.564645e-03,0.000218,0.000000
50%,0.490138,7.376503e-03,2.968826e-03,1.916816e-02,-1.176067e-03,-1.561300e-03,-3.740532e-03,3.325587e-04,1.117499e-03,-3.297772e-03,...,-1.082614e-03,6.457093e-04,-4.968362e-04,8.937860e-03,2.206703e-03,-1.482342e-02,4.245658e-05,3.321879e-04,0.000856,0.000000
75%,0.806290,5.359182e-01,3.643729e-02,1.094793e-01,4.404896e-02,1.758325e-02,5.437327e-03,4.730396e-03,1.636140e-02,3.829043e-02,...,6.851388e-03,5.032363e-02,6.553594e-03,9.587129e-02,4.664026e-02,6.850398e-02,2.880063e-03,2.312704e-03,0.003004,0.000000
max,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,1.000000


In [7]:
# Separando X e y

X = transacoes.drop(columns='Class')
y = transacoes['Class']

In [8]:
# Separando em treino e teste

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0,stratify=y)

### XGBoost 

In [9]:
# Criando o modelo XGBoost

modelo_xgb = xgb.XGBClassifier(
    max_depth=9,
    n_estimators=200,
    learning_rate=0.3,
    random_state=0,
    scale_pos_weight=(y.value_counts()[0] / y.value_counts()[1])  # Lida com desbalanceamento
)

# Treinando e avaliando o modelo

modelo_xgb.fit(X_train, y_train)

y_pred_xgb = modelo_xgb.predict(X_test)

# Calculando as métricas

acc_xgb = accuracy_score(y_test, y_pred_xgb)
prec_xgb = precision_score(y_test, y_pred_xgb)
rec_xgb = recall_score(y_test, y_pred_xgb)

# Vusualizando a matriz confusão

confusion_matrix(y_test, y_pred_xgb)

array([[93817,     8],
       [   34,   128]])

In [10]:
# Utilizando as métricas de avaliação

print('Acurácia:', accuracy_score(y_test, y_pred_xgb))
print('Precisão:', precision_score(y_test, y_pred_xgb))
print('Recall:', recall_score(y_test, y_pred_xgb, pos_label=1))

Acurácia: 0.9995531296881484
Precisão: 0.9411764705882353
Recall: 0.7901234567901234


### Random Forest

In [11]:
# Utilizando o Random Forest

clf_RF = RandomForestClassifier(
    max_depth=7,  
    random_state=0,
    criterion='gini',
    n_estimators=100,  
    class_weight='balanced_subsample' 
).fit(X_train, y_train)

y_pred_RF = clf_RF.predict(X_test)

# Vusualizando a matriz confusão

confusion_matrix(y_test, y_pred_RF)

array([[93787,    38],
       [   31,   131]])

In [12]:
# Utilizando as métricas de avaliação

print('Acurácia:', accuracy_score(y_test, y_pred_RF))
print('Precisão:', precision_score(y_test, y_pred_RF))
print('Recall:', recall_score(y_test, y_pred_RF))

Acurácia: 0.9992658559162437
Precisão: 0.7751479289940828
Recall: 0.808641975308642


### Regressão Logística

In [13]:
# Utilizando a Regressão Logística

clfLog = LogisticRegression(random_state=0).fit(X_train, y_train)

# Calculando as métricas

y_predLog = clfLog.predict(X_test)
y_pred_probaLog = clfLog.predict_proba(X_test)[:,1]

# Vusualizando a matriz confusão

confusion_matrix(y_test, y_predLog)

array([[93811,    14],
       [   70,    92]])

In [14]:
# Utilizando as métricas de avaliação

print('Acurácia:', accuracy_score(y_test, y_predLog))
print('Precisão:', precision_score(y_test, y_predLog))
print('Recall:', recall_score(y_test, y_predLog))

Acurácia: 0.9991062593762967
Precisão: 0.8679245283018868
Recall: 0.5679012345679012


### OBSERVAÇÃO:

**Utilizamos a base desbalanceada e fizemos uma espécie de 'normalização' nos dados**.

- Utilizando os três melhores algoritmos até aqui, temos como desempenho:
  
  - **1º Lugar**: XGBoost
  - **2º Lugar**: Random Forest
  - **3º Lugar**: Regressão Logística